In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

# Cesta k LoRA modelu
lora_path = "./final_synthetic_deepseek"

# Načtení konfigurace LoRA modelu
peft_config = PeftConfig.from_pretrained(lora_path)

# Načtení base modelu
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# Připojení LoRA adapteru
model = PeftModel.from_pretrained(base_model, lora_path)

# Načtení tokenizeru
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path, trust_remote_code=True)

# Zpracování CSV a vytvoření promptu
df = pd.read_csv("CSV-Hashed.csv")  
csv_text = df.to_csv(index=False) 
prompt = f"I will give you CSV and your job is to find frequent itemsets of maxlenght 5 with support 0.75. Do not use any code, it is your job to do!<csv>\n{csv_text}\n</csv>\n"

# Tokenizace vstupu
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generování odpovědi
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=5000)  # Zvolena vysoká hodnota pro možnost generování rozsáhlých odpovědí. Vyšší hodnota způsobovala zamrznutí modelu (nízká VRAM)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    temperature=1 # nebo 0, nebo 0.5

# Uložení výsledku
with open("model_output.txt", "w", encoding="utf-8") as f:
    f.write(decoded_output)

print("Výstup byl uložen do 'model_output.txt'")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


✅ Výstup byl uložen do 'model_output.txt'
